In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

%load_ext line_profiler

In [24]:
import random
from stewicombo import combineInventoriesforFacilitiesinOneInventory
from stewicombo.overlaphandler import aggregate_and_remove_overlap, reliablity_weighted_sum, join_with_underscore, get_first_item, get_by_preference
from electricitylci.globals import inventories_of_interest
import pandas as pd

In [37]:
df = pd.DataFrame({'a': ['a', 'a', 'b', 'c', 'd', 'e'], 
                   'b': range(6)})
INVENTORY_PREFERENCE_BY_COMPARTMENT = {"air":["eGRID","GHGRP","NEI","TRI"],
                                       "water":["DMR", "TRI"],
                                       "soil":["TRI"],
                                       "waste":["RCRAInfo","TRI"]}

In [27]:
%%timeit
random.choice(list(INVENTORY_PREFERENCE_BY_COMPARTMENT))

1.08 µs ± 20.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [34]:
df.head()

,a,b
0,a,0
1,a,1
2,b,2
3,b,3
4,c,4


In [31]:
def get_by_preference1(group):
    prefs = random.choice(list(INVENTORY_PREFERENCE_BY_COMPARTMENT))
    
    for pref in prefs:
        for index, row in group.iterrows():
            a = row['a']
        
def get_by_preference2(group):
    if len(group) > 1:
        prefs = random.choice(list(INVENTORY_PREFERENCE_BY_COMPARTMENT))
    
        for pref in prefs:
            for index, row in group.iterrows():
                a = index
    else:
        a = group['a']

In [38]:
%%timeit
df.groupby('a').apply(get_by_preference1)

5.05 ms ± 33.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [39]:
%%timeit
df.groupby('a').apply(get_by_preference2)

2.77 ms ± 53.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
inventories_of_interest

{'eGRID': '2014', 'TRI': '2014', 'NEI': '2014', 'RCRAInfo': '2013'}

In [22]:
inventories_of_interest.pop('RCRAInfo')

'2013'

In [35]:
%pdb

Automatic pdb calling has been turned ON


In [42]:
from stewicombo.globals import COL_FUNC_DEFAULT
COL_FUNC_DEFAULT

'first'

In [45]:
%lprun -f get_by_preference -f reliablity_weighted_sum -f aggregate_and_remove_overlap combineInventoriesforFacilitiesinOneInventory("eGRID",inventories_of_interest,filter_for_LCI=True)

Aggregating inventories...
Adding any rows with NaN FRS_ID or SRS_ID
Overlap removed.


/Users/greg/miniconda3/envs/elci/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Timer unit: 1e-06 s

Total time: 78.3378 s
File: /Users/greg/Documents/NETL work/ElectricityLCI/standardizedinventories/stewicombo/overlaphandler.py
Function: reliablity_weighted_sum at line 19

Line #      Hits         Time  Per Hit   % Time  Line Contents
    19                                           def reliablity_weighted_sum(df, weights_col_name, items):
    20     29285    6560013.0    224.0      8.4      grouped = df.groupby(SOURCE_COL)
    21                                           
    22     29285    2276333.0     77.7      2.9      for x, y in items.items():
    23     29285      29848.0      1.0      0.0          first_index = x
    24     29285      30034.0      1.0      0.0          break
    25                                           
    26                                               # group_name = df.iloc[first_index].loc[SOURCE_COL]
    27     29285    2228439.0     76.1      2.8      group_name = df.loc[first_index, SOURCE_COL]
    28     29285   44427639.0 

In [39]:
%lprun -f get_by_preference -f get_first_item -f reliablity_weighted_sum -f aggregate_and_remove_overlap combineInventoriesforFacilitiesinOneInventory("eGRID",inventories_of_interest,filter_for_LCI=True)

Aggregating inventories...
Adding any rows with NaN FRS_ID or SRS_ID
Overlap removed.


/Users/greg/miniconda3/envs/elci/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Timer unit: 1e-06 s

Total time: 73.3102 s
File: /Users/greg/Documents/NETL work/ElectricityLCI/standardizedinventories/stewicombo/overlaphandler.py
Function: reliablity_weighted_sum at line 19

Line #      Hits         Time  Per Hit   % Time  Line Contents
    19                                           def reliablity_weighted_sum(df, weights_col_name, items):
    20     29285    6170222.0    210.7      8.4      grouped = df.groupby(SOURCE_COL)
    21                                           
    22     29285    2103332.0     71.8      2.9      for x, y in items.items():
    23     29285      22331.0      0.8      0.0          first_index = x
    24     29285      26466.0      0.9      0.0          break
    25                                           
    26                                               # group_name = df.iloc[first_index].loc[SOURCE_COL]
    27     29285    2027171.0     69.2      2.8      group_name = df.loc[first_index, SOURCE_COL]
    28     29285   41703195.0 

In [6]:
%prun combineInventoriesforFacilitiesinOneInventory("eGRID",inventories_of_interest,filter_for_LCI=True)

Aggregating inventories...
Adding any rows with NaN FRS_ID or SRS_ID
Overlap removed.


/Users/greg/miniconda3/envs/elci/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


         2666407460 function calls (2632969518 primitive calls) in 2537.543 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
564625061  138.129    0.000  220.344    0.000 {built-in method builtins.isinstance}
3950240/3524751   57.010    0.000  245.669    0.000 base.py:255(__new__)
  1063640   50.285    0.000  455.990    0.000 {method 'get_result' of 'pandas._libs.reduction.SeriesGrouper' objects}
  2127280   49.748    0.000   49.748    0.000 {pandas._libs.lib.maybe_convert_objects}
127206159   49.052    0.000   70.412    0.000 generic.py:7(_check)
43227467/42901050   47.123    0.000   75.066    0.000 common.py:1835(_get_dtype_type)
156977166/125795504   45.294    0.000   61.132    0.000 {built-in method builtins.len}
6991991/6991979   40.912    0.000  411.704    0.000 series.py:166(__init__)
 58631438   40.021    0.000   72.829    0.000 base.py:61(is_dtype)
205782620/205569892   38.676    0.000   50.104    0.000 {built-in me